In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class DigitDataset(Dataset):
    def __init__(self, data_dir, transform=None, max_boxes=10):
        self.data_dir = data_dir
        self.transform = transform
        self.images = [img for img in os.listdir(data_dir) if img.endswith('.png')]
        self.max_boxes = max_boxes
        self.annotations = self._load_annotations()

    def _load_annotations(self):
        annotations = {}
        for img in self.images:
            annotation_path = os.path.join(self.data_dir, img.replace('.png', '.txt'))
            if os.path.exists(annotation_path):
                with open(annotation_path, 'r') as f:
                    boxes = []
                    for line in f:
                        boxes.append([float(x) for x in line.strip().split()])
                    annotations[img] = boxes
        return annotations

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.data_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        targets = self.annotations.get(img_name, [])
        # Pad targets to fixed size
        padded_targets = np.zeros((self.max_boxes, 5))
        if len(targets) > 0:
            targets = np.array(targets)
            padded_targets[:len(targets), :5] = targets

        padded_targets = torch.tensor(padded_targets, dtype=torch.float32)
        return image, padded_targets

train_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
])

class SmallResNet(nn.Module):
    def __init__(self, num_boxes=10):
        super(SmallResNet, self).__init__()
        self.num_boxes = num_boxes
        self.model = models.resnet18(pretrained=True)
        # Modify the first convolutional layer to accept 32x32 images
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()  # Remove the first maxpool layer
        self.model.fc = nn.Linear(self.model.fc.in_features, num_boxes * 5)  # 5 predictions (class + 4 coordinates) per box

    def forward(self, x):
        return self.model(x).view(-1, self.num_boxes, 5)

model = SmallResNet().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, criterion, optimizer, dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, targets) in enumerate(dataloader):
            images = images.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            if i % 10 == 0:  # Display loss every 10 batches
                print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss:.4f}')

train_dataset = DigitDataset(data_dir='HW6_train', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

train_model(model, criterion, optimizer, train_loader, num_epochs=20)

def predict(model, dataloader):
    model.eval()
    results = []
    with torch.no_grad():
        for idx, (image, _) in enumerate(dataloader):
            image = image.to(device)
            outputs = model(image)
            outputs = outputs.cpu().numpy()
            for i, output in enumerate(outputs):
                for box in output:
                    cls = int(box[0])
                    x = box[1]
                    y = box[2]
                    results.append([idx, cls, x, y])
    return results

test_dataset = DigitDataset(data_dir='HW6_test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

predictions = predict(model, test_loader)

# Convert predictions to dataframe
df = pd.DataFrame(predictions, columns=['IMAGE_index', 'class', 'x', 'y'])
df.to_csv('predictions.csv', index=False)


C:\Users\gg750\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\gg750\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/20], Batch [1/3000], Loss: 2.5314
Epoch [1/20], Batch [11/3000], Loss: 0.6009
Epoch [1/20], Batch [21/3000], Loss: 0.7004
Epoch [1/20], Batch [31/3000], Loss: 0.5076
Epoch [1/20], Batch [41/3000], Loss: 0.6256
Epoch [1/20], Batch [51/3000], Loss: 0.5802
Epoch [1/20], Batch [61/3000], Loss: 0.4676
Epoch [1/20], Batch [71/3000], Loss: 0.5023
Epoch [1/20], Batch [81/3000], Loss: 0.4553
Epoch [1/20], Batch [91/3000], Loss: 0.4810
Epoch [1/20], Batch [101/3000], Loss: 0.4509
Epoch [1/20], Batch [111/3000], Loss: 0.4664
Epoch [1/20], Batch [121/3000], Loss: 0.3765
Epoch [1/20], Batch [131/3000], Loss: 0.4829
Epoch [1/20], Batch [141/3000], Loss: 0.4884
Epoch [1/20], Batch [151/3000], Loss: 0.4624
Epoch [1/20], Batch [161/3000], Loss: 0.4929
Epoch [1/20], Batch [171/3000], Loss: 0.4286
Epoch [1/20], Batch [181/3000], Loss: 0.4778
Epoch [1/20], Batch [191/3000], Loss: 0.3923
Epoch [1/20], Batch [201/3000], Loss: 0.3833
Epoch [1/20], Batch [211/3000], Loss: 0.3962
Epoch [1/20], Batch [

Epoch [1/20], Batch [1811/3000], Loss: 0.3162
Epoch [1/20], Batch [1821/3000], Loss: 0.3647
Epoch [1/20], Batch [1831/3000], Loss: 0.3598
Epoch [1/20], Batch [1841/3000], Loss: 0.3783
Epoch [1/20], Batch [1851/3000], Loss: 0.3888
Epoch [1/20], Batch [1861/3000], Loss: 0.4614
Epoch [1/20], Batch [1871/3000], Loss: 0.5056
Epoch [1/20], Batch [1881/3000], Loss: 0.4434
Epoch [1/20], Batch [1891/3000], Loss: 0.3504
Epoch [1/20], Batch [1901/3000], Loss: 0.3005
Epoch [1/20], Batch [1911/3000], Loss: 0.3661
Epoch [1/20], Batch [1921/3000], Loss: 0.4163
Epoch [1/20], Batch [1931/3000], Loss: 0.3850
Epoch [1/20], Batch [1941/3000], Loss: 0.3052
Epoch [1/20], Batch [1951/3000], Loss: 0.3544
Epoch [1/20], Batch [1961/3000], Loss: 0.3837
Epoch [1/20], Batch [1971/3000], Loss: 0.3074
Epoch [1/20], Batch [1981/3000], Loss: 0.4841
Epoch [1/20], Batch [1991/3000], Loss: 0.4594
Epoch [1/20], Batch [2001/3000], Loss: 0.5451
Epoch [1/20], Batch [2011/3000], Loss: 0.2917
Epoch [1/20], Batch [2021/3000], L

Epoch [2/20], Batch [601/3000], Loss: 0.2710
Epoch [2/20], Batch [611/3000], Loss: 0.3955
Epoch [2/20], Batch [621/3000], Loss: 0.3125
Epoch [2/20], Batch [631/3000], Loss: 0.4318
Epoch [2/20], Batch [641/3000], Loss: 0.3624
Epoch [2/20], Batch [651/3000], Loss: 0.3541
Epoch [2/20], Batch [661/3000], Loss: 0.3367
Epoch [2/20], Batch [671/3000], Loss: 0.3454
Epoch [2/20], Batch [681/3000], Loss: 0.4420
Epoch [2/20], Batch [691/3000], Loss: 0.2866
Epoch [2/20], Batch [701/3000], Loss: 0.3780
Epoch [2/20], Batch [711/3000], Loss: 0.3660
Epoch [2/20], Batch [721/3000], Loss: 0.2926
Epoch [2/20], Batch [731/3000], Loss: 0.3897
Epoch [2/20], Batch [741/3000], Loss: 0.4113
Epoch [2/20], Batch [751/3000], Loss: 0.3562
Epoch [2/20], Batch [761/3000], Loss: 0.3137
Epoch [2/20], Batch [771/3000], Loss: 0.4076
Epoch [2/20], Batch [781/3000], Loss: 0.3085
Epoch [2/20], Batch [791/3000], Loss: 0.4363
Epoch [2/20], Batch [801/3000], Loss: 0.4279
Epoch [2/20], Batch [811/3000], Loss: 0.4068
Epoch [2/2

Epoch [2/20], Batch [2391/3000], Loss: 0.4320
Epoch [2/20], Batch [2401/3000], Loss: 0.5048
Epoch [2/20], Batch [2411/3000], Loss: 0.3352
Epoch [2/20], Batch [2421/3000], Loss: 0.3568
Epoch [2/20], Batch [2431/3000], Loss: 0.3401
Epoch [2/20], Batch [2441/3000], Loss: 0.3887
Epoch [2/20], Batch [2451/3000], Loss: 0.4477
Epoch [2/20], Batch [2461/3000], Loss: 0.2649
Epoch [2/20], Batch [2471/3000], Loss: 0.5126
Epoch [2/20], Batch [2481/3000], Loss: 0.3491
Epoch [2/20], Batch [2491/3000], Loss: 0.4435
Epoch [2/20], Batch [2501/3000], Loss: 0.4651
Epoch [2/20], Batch [2511/3000], Loss: 0.4538
Epoch [2/20], Batch [2521/3000], Loss: 0.3916
Epoch [2/20], Batch [2531/3000], Loss: 0.3560
Epoch [2/20], Batch [2541/3000], Loss: 0.5109
Epoch [2/20], Batch [2551/3000], Loss: 0.3896
Epoch [2/20], Batch [2561/3000], Loss: 0.3154
Epoch [2/20], Batch [2571/3000], Loss: 0.4468
Epoch [2/20], Batch [2581/3000], Loss: 0.4210
Epoch [2/20], Batch [2591/3000], Loss: 0.4107
Epoch [2/20], Batch [2601/3000], L

...

Epoch [18/20], Batch [531/3000], Loss: 0.1422
Epoch [18/20], Batch [541/3000], Loss: 0.1307
Epoch [18/20], Batch [551/3000], Loss: 0.1169
Epoch [18/20], Batch [561/3000], Loss: 0.1460
Epoch [18/20], Batch [571/3000], Loss: 0.1315
Epoch [18/20], Batch [581/3000], Loss: 0.1692
Epoch [18/20], Batch [591/3000], Loss: 0.1519
Epoch [18/20], Batch [601/3000], Loss: 0.0929
Epoch [18/20], Batch [611/3000], Loss: 0.0958
Epoch [18/20], Batch [621/3000], Loss: 0.1294
Epoch [18/20], Batch [631/3000], Loss: 0.0934
Epoch [18/20], Batch [641/3000], Loss: 0.1335
Epoch [18/20], Batch [651/3000], Loss: 0.1607
Epoch [18/20], Batch [661/3000], Loss: 0.1040
Epoch [18/20], Batch [671/3000], Loss: 0.0669
Epoch [18/20], Batch [681/3000], Loss: 0.0955
Epoch [18/20], Batch [691/3000], Loss: 0.1263
Epoch [18/20], Batch [701/3000], Loss: 0.1116
Epoch [18/20], Batch [711/3000], Loss: 0.1443
Epoch [18/20], Batch [721/3000], Loss: 0.1206
Epoch [18/20], Batch [731/3000], Loss: 0.0900
Epoch [18/20], Batch [741/3000], L

Epoch [18/20], Batch [2291/3000], Loss: 0.1574
Epoch [18/20], Batch [2301/3000], Loss: 0.1644
Epoch [18/20], Batch [2311/3000], Loss: 0.1701
Epoch [18/20], Batch [2321/3000], Loss: 0.0747
Epoch [18/20], Batch [2331/3000], Loss: 0.1663
Epoch [18/20], Batch [2341/3000], Loss: 0.1507
Epoch [18/20], Batch [2351/3000], Loss: 0.1259
Epoch [18/20], Batch [2361/3000], Loss: 0.1326
Epoch [18/20], Batch [2371/3000], Loss: 0.1599
Epoch [18/20], Batch [2381/3000], Loss: 0.1368
Epoch [18/20], Batch [2391/3000], Loss: 0.1251
Epoch [18/20], Batch [2401/3000], Loss: 0.1141
Epoch [18/20], Batch [2411/3000], Loss: 0.1577
Epoch [18/20], Batch [2421/3000], Loss: 0.1269
Epoch [18/20], Batch [2431/3000], Loss: 0.1169
Epoch [18/20], Batch [2441/3000], Loss: 0.1165
Epoch [18/20], Batch [2451/3000], Loss: 0.1342
Epoch [18/20], Batch [2461/3000], Loss: 0.1389
Epoch [18/20], Batch [2471/3000], Loss: 0.1676
Epoch [18/20], Batch [2481/3000], Loss: 0.1207
Epoch [18/20], Batch [2491/3000], Loss: 0.1171
Epoch [18/20]

Epoch [19/20], Batch [1051/3000], Loss: 0.0960
Epoch [19/20], Batch [1061/3000], Loss: 0.0890
Epoch [19/20], Batch [1071/3000], Loss: 0.0770
Epoch [19/20], Batch [1081/3000], Loss: 0.1403
Epoch [19/20], Batch [1091/3000], Loss: 0.0861
Epoch [19/20], Batch [1101/3000], Loss: 0.0775
Epoch [19/20], Batch [1111/3000], Loss: 0.0931
Epoch [19/20], Batch [1121/3000], Loss: 0.0726
Epoch [19/20], Batch [1131/3000], Loss: 0.0967
Epoch [19/20], Batch [1141/3000], Loss: 0.0805
Epoch [19/20], Batch [1151/3000], Loss: 0.1032
Epoch [19/20], Batch [1161/3000], Loss: 0.0965
Epoch [19/20], Batch [1171/3000], Loss: 0.1082
Epoch [19/20], Batch [1181/3000], Loss: 0.1220
Epoch [19/20], Batch [1191/3000], Loss: 0.1210
Epoch [19/20], Batch [1201/3000], Loss: 0.1133
Epoch [19/20], Batch [1211/3000], Loss: 0.0997
Epoch [19/20], Batch [1221/3000], Loss: 0.1019
Epoch [19/20], Batch [1231/3000], Loss: 0.1340
Epoch [19/20], Batch [1241/3000], Loss: 0.1519
Epoch [19/20], Batch [1251/3000], Loss: 0.0967
Epoch [19/20]

Epoch [19/20], Batch [2801/3000], Loss: 0.1650
Epoch [19/20], Batch [2811/3000], Loss: 0.1344
Epoch [19/20], Batch [2821/3000], Loss: 0.1041
Epoch [19/20], Batch [2831/3000], Loss: 0.1119
Epoch [19/20], Batch [2841/3000], Loss: 0.0921
Epoch [19/20], Batch [2851/3000], Loss: 0.0866
Epoch [19/20], Batch [2861/3000], Loss: 0.1428
Epoch [19/20], Batch [2871/3000], Loss: 0.1234
Epoch [19/20], Batch [2881/3000], Loss: 0.1047
Epoch [19/20], Batch [2891/3000], Loss: 0.0844
Epoch [19/20], Batch [2901/3000], Loss: 0.1166
Epoch [19/20], Batch [2911/3000], Loss: 0.1417
Epoch [19/20], Batch [2921/3000], Loss: 0.1154
Epoch [19/20], Batch [2931/3000], Loss: 0.0950
Epoch [19/20], Batch [2941/3000], Loss: 0.1302
Epoch [19/20], Batch [2951/3000], Loss: 0.1020
Epoch [19/20], Batch [2961/3000], Loss: 0.1766
Epoch [19/20], Batch [2971/3000], Loss: 0.1189
Epoch [19/20], Batch [2981/3000], Loss: 0.1146
Epoch [19/20], Batch [2991/3000], Loss: 0.1074
Epoch 19/20, Average Loss: 0.1071
Epoch [20/20], Batch [1/30

Epoch [20/20], Batch [1561/3000], Loss: 0.0842
Epoch [20/20], Batch [1571/3000], Loss: 0.0822
Epoch [20/20], Batch [1581/3000], Loss: 0.0981
Epoch [20/20], Batch [1591/3000], Loss: 0.0789
Epoch [20/20], Batch [1601/3000], Loss: 0.0886
Epoch [20/20], Batch [1611/3000], Loss: 0.1478
Epoch [20/20], Batch [1621/3000], Loss: 0.0859
Epoch [20/20], Batch [1631/3000], Loss: 0.0832
Epoch [20/20], Batch [1641/3000], Loss: 0.0750
Epoch [20/20], Batch [1651/3000], Loss: 0.0781
Epoch [20/20], Batch [1661/3000], Loss: 0.0932
Epoch [20/20], Batch [1671/3000], Loss: 0.0905
Epoch [20/20], Batch [1681/3000], Loss: 0.0773
Epoch [20/20], Batch [1691/3000], Loss: 0.0799
Epoch [20/20], Batch [1701/3000], Loss: 0.1154
Epoch [20/20], Batch [1711/3000], Loss: 0.0662
Epoch [20/20], Batch [1721/3000], Loss: 0.1102
Epoch [20/20], Batch [1731/3000], Loss: 0.0943
Epoch [20/20], Batch [1741/3000], Loss: 0.0998
Epoch [20/20], Batch [1751/3000], Loss: 0.1170
Epoch [20/20], Batch [1761/3000], Loss: 0.0642
Epoch [20/20]

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class DigitDataset(Dataset):
    def __init__(self, data_dir, transform=None, max_boxes=10):
        self.data_dir = data_dir
        self.transform = transform
        self.images = [img for img in os.listdir(data_dir) if img.endswith('.png')]
        self.max_boxes = max_boxes
        self.annotations = self._load_annotations()

    def _load_annotations(self):
        annotations = {}
        for img in self.images:
            annotation_path = os.path.join(self.data_dir, img.replace('.png', '.txt'))
            if os.path.exists(annotation_path):
                with open(annotation_path, 'r') as f:
                    boxes = []
                    for line in f:
                        boxes.append([float(x) for x in line.strip().split()])
                    annotations[img] = boxes
        return annotations

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.data_dir, img_name)
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        targets = self.annotations.get(img_name, [])
        # Pad targets to fixed size
        padded_targets = np.zeros((self.max_boxes, 5))
        if len(targets) > 0:
            targets = np.array(targets)
            padded_targets[:len(targets), :5] = targets

        padded_targets = torch.tensor(padded_targets, dtype=torch.float32)
        return image, padded_targets

train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

class DigitDetector(nn.Module):
    def __init__(self, num_boxes=10):
        super(DigitDetector, self).__init__()
        self.num_boxes = num_boxes
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_boxes * 5)  # 5 predictions (class + 4 coordinates) per box

    def forward(self, x):
        return self.model(x).view(-1, self.num_boxes, 5)

model = DigitDetector().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, criterion, optimizer, dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for i, (images, targets) in enumerate(dataloader):
            images = images.to(device)
            targets = targets.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            if i % 10 == 0:  # Display loss every 10 batches
                print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss:.4f}')

train_dataset = DigitDataset(data_dir='HW6_train', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)

train_model(model, criterion, optimizer, train_loader, num_epochs=1)###

def predict(model, dataloader):
    model.eval()
    results = []
    with torch.no_grad():
        for idx, (image, _) in enumerate(dataloader):
            image = image.to(device)
            outputs = model(image)
            outputs = outputs.cpu().numpy()
            for i, output in enumerate(outputs):
                for box in output:
                    cls = int(box[0])
                    x = box[1]
                    y = box[2]
                    results.append([idx, cls, x, y])
    return results

test_dataset = DigitDataset(data_dir='HW6_test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

predictions = predict(model, test_loader)

# Convert predictions to dataframe
df = pd.DataFrame(predictions, columns=['IMAGE_index', 'class', 'x', 'y'])
df.to_csv('predictions.csv', index=False)


C:\Users\gg750\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\gg750\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision.transforms as transforms
# import torchvision.datasets as datasets
# import torchvision.models as models
# from torch.utils.data import DataLoader, Dataset
# from PIL import Image
# import os
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# class DigitDataset(Dataset):
#     def __init__(self, data_dir, transform=None, max_boxes=10):
#         self.data_dir = data_dir
#         self.transform = transform
#         self.images = [img for img in os.listdir(data_dir) if img.endswith('.png')]
#         self.max_boxes = max_boxes
#         self.annotations = self._load_annotations()

#     def _load_annotations(self):
#         annotations = {}
#         for img in self.images:
#             annotation_path = os.path.join(self.data_dir, img.replace('.png', '.txt'))
#             if os.path.exists(annotation_path):
#                 with open(annotation_path, 'r') as f:
#                     boxes = []
#                     for line in f:
#                         boxes.append([float(x) for x in line.strip().split()])
#                     annotations[img] = boxes
#         return annotations

#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         img_name = self.images[idx]
#         img_path = os.path.join(self.data_dir, img_name)
#         image = Image.open(img_path).convert("RGB")

#         if self.transform:
#             image = self.transform(image)

#         targets = self.annotations.get(img_name, [])
#         # Pad targets to fixed size
#         padded_targets = np.zeros((self.max_boxes, 5))
#         if len(targets) > 0:
#             targets = np.array(targets)
#             padded_targets[:len(targets), :5] = targets

#         padded_targets = torch.tensor(padded_targets, dtype=torch.float32)
#         return image, padded_targets

# train_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])

# test_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
# ])


# class DigitDetector(nn.Module):
#     def __init__(self, num_boxes=10):
#         super(DigitDetector, self).__init__()
#         self.num_boxes = num_boxes
#         self.model = models.resnet18(pretrained=True)
#         self.model.fc = nn.Linear(self.model.fc.in_features, num_boxes * 5)  # 5 predictions (class + 4 coordinates) per box

#     def forward(self, x):
#         return self.model(x).view(-1, self.num_boxes, 5)

# model = DigitDetector().to(device)


# criterion = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)


# def train_model(model, criterion, optimizer, dataloader, num_epochs):
#     model.train()
#     for epoch in range(num_epochs):
#         running_loss = 0.0
#         for i, (images, targets) in enumerate(dataloader):
#             images = images.to(device)
#             targets = targets.to(device)  # No need to reshape here

#             optimizer.zero_grad()
#             outputs = model(images)
            
#             # Reshape targets and outputs to match the loss function expectations
#             loss = criterion(outputs, targets)
#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item() * images.size(0)
#             if i % 10 == 0:  # Display loss every 10 batches
#                 print(f'Epoch [{epoch+1}/{num_epochs}], Batch [{i+1}/{len(dataloader)}], Loss: {loss.item():.4f}')

#         epoch_loss = running_loss / len(dataloader.dataset)
#         print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {epoch_loss:.4f}')

# train_dataset = DigitDataset(data_dir='HW6_train', transform=train_transform)
# # train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

# train_model(model, criterion, optimizer, train_loader, num_epochs=1)###



# def predict(model, dataloader):
#     model.eval()
#     results = []
#     with torch.no_grad():
#         for idx, image in enumerate(dataloader):
#             image = image.to(device)
#             outputs = model(image)
#             outputs = outputs.cpu().numpy()
#             for output in outputs:
#                 cls = int(output[0])
#                 x = output[1]
#                 y = output[2]
#                 results.append([idx, cls, x, y])
#     return results

# test_dataset = DigitDataset(data_dir='HW6_test', transform=test_transform)
# test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# predictions = predict(model, test_loader)

# # Convert predictions to dataframe
# df = pd.DataFrame(predictions, columns=['IMAGE_index', 'class', 'x', 'y'])
# df.to_csv('predictions.csv', index=False)